In [4]:
!pip install datasets
!pip install transformers
!pip install tokenizers
!pip install torch
!pip install scikit-learn

  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 11.2 MB/s eta 0:00:0000:01:01
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.4/22.4 MB 40.5 MB/s eta 0:00:00a 0:00:01


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from datasets import load_dataset
from transformers import AutoTokenizer, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

/Users/andrespuertagonzalez/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.pre_tokenizers import Whitespace
from tokenizers import Tokenizer, models, trainers, pre_tokenizers, processors

In [4]:
from transformers import PreTrainedTokenizerFast

In [5]:
# Load AG_NEWS dataset using the datasets library
dataset = load_dataset("ag_news")
dataset_train = dataset["train"]
dataset_test = dataset["test"]

In [6]:
len(dataset_train)

120000

In [7]:
# Initialize a BPE tokenizer
tokenizer = Tokenizer(models.BPE(unk_token="[UNK]"))

# Set the pre-tokenizer to split on whitespace
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()

# Set up the trainer with special tokens
trainer = trainers.BpeTrainer(
    vocab_size=1000,
    special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"]
)

# Define a generator to yield batches of text
def batch_iterator(batch_size=1000):
    for i in range(0, len(dataset_train), batch_size):
        yield dataset_train[i:i + batch_size]["text"]

# Train the tokenizer
tokenizer.train_from_iterator(batch_iterator(), trainer=trainer)

# Set the post-processor to add special tokens
tokenizer.post_processor = processors.TemplateProcessing(
    single="[CLS] $A [SEP]",
    pair="[CLS] $A [SEP] $B:1 [SEP]:1",
    special_tokens=[
        ("[CLS]", tokenizer.token_to_id("[CLS]")),
        ("[SEP]", tokenizer.token_to_id("[SEP]")),
    ],
)
# Enable truncation and padding
tokenizer.enable_truncation(max_length=128)
tokenizer.enable_padding(pad_id=tokenizer.token_to_id("[PAD]"), pad_token="[PAD]", length=128)

In [8]:
tokenizer.save("custom_tokenizer.json")

In [9]:
fast_tokenizer = PreTrainedTokenizerFast(tokenizer_file="custom_tokenizer.json")

In [10]:
def preprocess_data(examples):
    # Tokenize the text
    encodings = tokenizer.encode_batch(examples["text"])

    # Extract input IDs and attention masks
    input_ids = [encoding.ids for encoding in encodings]
    attention_masks = [encoding.attention_mask for encoding in encodings]

    return {
        "input_ids": input_ids,
        "attention_mask": attention_masks,
        "label": examples["label"]
    }


In [11]:
# Apply preprocessing
ds_train = dataset_train.map(preprocess_data, batched=True)
ds_test = dataset_test.map(preprocess_data, batched=True)

In [12]:
ds_train[0]

{'text': "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.",
 'label': 2,
 'input_ids': [1,
  54,
  150,
  202,
  16,
  33,
  996,
  34,
  699,
  33,
  215,
  157,
  99,
  95,
  923,
  215,
  11,
  219,
  12,
  219,
  15,
  383,
  692,
  15,
  78,
  245,
  133,
  14,
  54,
  150,
  202,
  104,
  124,
  10,
  78,
  63,
  82,
  286,
  71,
  101,
  58,
  61,
  112,
  103,
  745,
  144,
  15,
  62,
  84,
  73,
  605,
  14,
  192,
  151,
  64,
  101,
  66,
  104,
  96,
  362,
  16,
  2,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  

In [13]:
# Define Basic Embedding Model
class BasicEmbeddingModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_classes):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=3)
        self.fc = nn.Linear(embed_dim, num_classes)

    def forward(self, input_ids, attention_mask, labels=None):
        embedded = self.embedding(input_ids)
        embedded = torch.mean(embedded, dim=1)  # Averaging embeddings
        logits = self.fc(embedded)

        loss = None
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(logits, labels)

        return {"loss": loss, "logits": logits} if loss is not None else {"logits": logits}

In [14]:
# Model setup
embed_dim = 100
num_classes = 4
vocab_size = 1000
model = BasicEmbeddingModel(vocab_size, embed_dim, num_classes)
# Show model summary
print(model)

BasicEmbeddingModel(
  (embedding): Embedding(1000, 100, padding_idx=3)
  (fc): Linear(in_features=100, out_features=4, bias=True)
)


In [15]:
# Define compute metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted')
    return {"accuracy": acc, "f1": f1}

In [16]:

# Define Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
)

In [17]:
# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds_train,
    eval_dataset=ds_test,
    processing_class=fast_tokenizer,
    compute_metrics=compute_metrics
)

In [18]:
# Train model
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.483400,0.483327,0.831711,0.831137
2,0.432300,0.446807,0.843816,0.843414
3,0.425600,0.440676,0.845000,0.844646


TrainOutput(global_step=11250, training_loss=0.5115090135362413, metrics={'train_runtime': 64.1468, 'train_samples_per_second': 5612.125, 'train_steps_per_second': 175.379, 'total_flos': 0.0, 'train_loss': 0.5115090135362413, 'epoch': 3.0})

# Lab: Enhancing a Basic Embedding Model with Positional Encoding and Multi-Head Attention

## Objective
In this lab, you will modify a basic sentiment analysis model by adding two key components from transformer-based architectures: **positional encoding** and **multi-head attention**. These modifications will improve the model’s ability to capture word order and token interactions, making it more expressive.

---

## Task Overview
You are provided with a basic embedding model that performs sentiment analysis by:
- Mapping token IDs to embeddings.
- Averaging embeddings across tokens.
- Passing the result through a fully connected layer for classification.

Your tasks:
1. **Implement positional encoding** to enrich word embeddings with information about token positions.
2. **Replace the mean pooling operation** with a **multi-head self-attention layer** to allow tokens to attend to each other.

---

## Step 1: Implement Positional Encoding
Transformers lack recurrence, so they rely on positional encodings to incorporate token order. You will implement **sinusoidal positional encoding**, as described in Vaswani et al. (2017), and add it to the input embeddings.

### Requirements
- Implement a function to generate sinusoidal positional encodings.
- Ensure the encoding is applied correctly to all input embeddings.

---

## Step 2: Implement Multi-Head Self-Attention
Instead of simply averaging word embeddings, you will replace this operation with **multi-head self-attention** to allow interactions between tokens.

### Requirements
- Implement a multi-head self-attention layer.
- Replace the `torch.mean(embedded, dim=1)` operation with the **attention mechanism**.
- Ensure that the implementation correctly processes padded tokens.

---


## Starter Code
Below is the base implementation of the model. You must complete the missing parts and train the model.

```python
import torch
import torch.nn as nn
import torch.nn.functional as F

class BasicEmbeddingModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_classes, num_heads):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=3)
        
        # Task 1: Implement positional encoding
        self.positional_encoding = PositionalEncoding(embed_dim)

        # Task 2: Implement Multi-Head Attention (replace mean pooling)
        # self.multihead_attn = ???

        self.fc = nn.Linear(embed_dim, num_classes)

    def forward(self, input_ids, attention_mask, labels=None):
        embedded = self.embedding(input_ids)
        
        # Apply positional encoding
        embedded = self.positional_encoding(embedded)

        # Task: Replace mean pooling with Multi-Head Attention
        # attn_output, _ = ???(embedded, embedded, embedded, key_padding_mask=attention_mask)
        
        # Pooling: Extract the first token's representation (CLS token equivalent)
        # pooled_output = attn_output[:, 0, :]

        logits = self.fc(pooled_output)

        loss = None
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(logits, labels)

        return {"loss": loss, "logits": logits} if loss is not None else {"logits": logits}

# Students must implement this class
class PositionalEncoding(nn.Module):
    def __init__(self, embed_dim, max_len=5000):
        super().__init__()
        self.encoding = self.create_positional_encoding(embed_dim, max_len)
        
    def create_positional_encoding(self, embed_dim, max_len):
        # Task: Implement sinusoidal positional encoding here
        pass

    def forward(self, x):
        # Task: Add positional encoding to embeddings
        pass
```

## Evaluation Criteria

To verify your implementation:

- Ensure that positional encoding correctly modifies embeddings.

- Confirm that the self-attention layer replaces mean pooling effectively.

- Train the modified model on a small sentiment dataset and compare results.

## Extra Challenges

For advanced students:

- Experiment with different pooling strategies (e.g., max pooling, CLS token extraction).

- Compare learned positional embeddings vs. sinusoidal encodings.

- Add layer normalization after attention.
